In [1]:
from astropy.visualization import AsymmetricPercentileInterval, ZScaleInterval
from astropy.io import fits
from astropy.wcs import WCS

from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle

import numpy as np
import os.path
import glob
import cv2
import sep

import query

from astroquery.gaia import Gaia
Gaia.ROW_LIMIT = -1

In [ ]:
def gaia_star_mask(img, wcs, pix=0.168, mask_a=694.7, mask_b=4.04,
                   size_buffer=1.4, gaia_bright=18.0,
                   factor_b=1.3, factor_f=1.9):
    """Find stars using Gaia and mask them out if necessary.

    Using the stars found in the GAIA TAP catalog, we build a bright star mask following
    similar procedure in Coupon et al. (2017).

    We separate the GAIA stars into bright (G <= 18.0) and faint (G > 18.0) groups, and
    apply different parameters to build the mask.
    """
    gaia_stars = query.image_gaia_stars(
        img, wcs, pixel=pix, mask_a=mask_a, mask_b=mask_b, verbose=False, visual=False,
        size_buffer=size_buffer)

    # Make a mask image
    msk_star = np.zeros(img.shape).astype('uint8')

    if gaia_stars is not None:
        gaia_b = gaia_stars[gaia_stars['phot_g_mean_mag'] <= gaia_bright]
        sep.mask_ellipse(msk_star, gaia_b['x_pix'], gaia_b['y_pix'],
                         gaia_b['rmask_arcsec'] / factor_b / pix,
                         gaia_b['rmask_arcsec'] / factor_b / pix, 0.0, r=1.0)

        gaia_f = gaia_stars[gaia_stars['phot_g_mean_mag'] > gaia_bright]
        sep.mask_ellipse(msk_star, gaia_f['x_pix'], gaia_f['y_pix'],
                         gaia_f['rmask_arcsec'] / factor_f / pix,
                         gaia_f['rmask_arcsec'] / factor_f / pix, 0.0, r=1.0)

        return gaia_stars, msk_star
    
def gaia2mask(filename):
    # read and scale the image
    ext = os.path.splitext(filename)[1]
    if ext == '.fz':
        hdu = fits.open(filename)[1]
    else:
        hdu = fits.open(filename)[0]    

    data = hdu.data
    header = hdu.header
    wcs = WCS(header)
    
    scale = 0.55 # SPLUS scale 
    gaia_stars, msk_star = gaia_star_mask ( data, wcs, pix = scale, mask_a = 694.7, mask_b = 4, 
                                           size_buffer = 1.8, gaia_bright = 24.0, factor_b = 1.4, factor_f = 1.9 )
    
    mask = np.multiply(data, (~msk_star.astype(bool)))   
    mask_file = os.path.splitext(filename)[0] + '_GMASK' + ext
    
    if ext == '.fz':
        imageHeader = fits.Header()
        hdu = fits.CompImageHDU ( mask, header )
        hdu.writeto ( mask_file, overwrite = True )
    else:
        fits.writeto ( mask_file, mask, hdu.header, overwrite = True ) 

    print ( '[+] File created: ', mask_file )

In [ ]:
filename = 'images/SPLUS-s27s34_R_swp.fz'

gaia2mask(filename)

In [ ]:
filename = 'images/SPLUS-s27s34_R_swp_GMASK.fz'

ext = os.path.splitext(filename)[1]
if ext == '.fz':
    hdu = fits.open(filename)[1]
else:
    hdu = fits.open(filename)[0]    

data = hdu.data
header = hdu.header
wcs = WCS(header)

zscale = ZScaleInterval(contrast=0.35) 
vmin, vmax = AsymmetricPercentileInterval ( lower_percentile = 50, upper_percentile = 95 ).get_limits ( zscale(data) )
plt.imsave ( fname = 'images/SPLUS-s27s34_R_swp_GMASK.png', arr = zscale(data), cmap = 'gray_r', 
            vmin = vmin, vmax = vmax, origin = 'lower', format = 'png' )